## **Set up ML Model**

In [0]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

In [0]:
!pip3 list | grep tensorflow
!pip3 install tensorflow==1.13.1 # the binary is still in the 1.13.1 version

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing   import LabelBinarizer
from tensorflow              import keras
from keras.callbacks         import ModelCheckpoint
from keras.models            import Sequential
from keras.layers            import Dense, Flatten, Conv2D, Dropout
from os.path                 import join
from keras.optimizers        import Adam

import keras.backend     as K
import matplotlib.pyplot as plt
import tensorflow        as tf
import numpy             as np
import pandas            as pd
import seaborn           as sb
import pdb, os, random, keras, re


Using TensorFlow backend.


## Load Data


In [0]:
def xNormalize(value, max_value, min_value):
    return (value - min_value)/(max_value - min_value)

def yNormalize(value, max_value):
    if value < 0:
        if max_value == 0:
            return -1
        else:
            # return max_value*(-1)
            return -1
    
    return value

def load_data2(directory):
    all_files = [join(directory, filename) for filename in os.listdir(directory)]
    
    random.shuffle(all_files)
    
    first_file = all_files.pop(0) 
    all_data   = pd.read_csv(first_file)
    
    for file in all_files:
        data = pd.read_csv(file)
        
        all_data = all_data.append(data)
        
    return all_data

def load_data(filepath):
    data = pd.read_csv(filepath)
    
    for i in range(3):
        data = data.sample(frac=1)
    
    return data

def get_coordID_values(num_nodes):
    values = []
    
    for i in range(1, number_of_nodes + 1):
        values.append('CoordID_' + str(i))
        
    return values
    
def get_estimatePeerID_values(num_nodes):
    values = []
    
    for i in range(1, number_of_nodes + 1):
        values.append('EstimatePeerID_' + str(i))
        
    return values

vote    = 20
coordID = 1
peerID  = 200
maj     = 10
deci    = 1

def desnormalize(data_norm):
    for column in data_norm.columns:
        if ('PeerID_' in column):
            data_norm[column] = data_norm[column].apply(lambda x: x * peerID)
        elif ('Decision' in column):
            data_norm[column] = data_norm[column].apply(lambda x: x * deci)    
        elif ('Vote' in column):
            data_norm[column] = data_norm[column].apply(lambda x: x * vote)
        elif ('Maj' in column):
            data_norm[column] = data_norm[column].apply(lambda x: x * maj)
        elif ('CoordID' in column):
            data_norm[column] = data_norm[column].apply(lambda x: x * coordID)
            
    return data_norm

In [0]:
number_of_nodes          = 40
algorithm_variables      = 8
matrix_rows, matrix_cols = 1, (algorithm_variables - 4) + number_of_nodes*4 + 2 

filepath = 'sample_data/snapshots_' + str(number_of_nodes)  + '.csv'
data     = load_data(filepath)

train_set_size = int(len(data) * 0.8)

data_np = data.to_numpy()
delay   = data_np[:, (algorithm_variables + number_of_nodes):]

max_val_delay = np.amax(delay)
min_val_delay = np.amin(delay)

data_norm = pd.get_dummies(data, 
                           columns=['EstimateValue', 'Decision', 'PeerID', 'Phase', 'EstimatePeerID', 'CoordID'],
                           prefix=['EstimateValue', 'Decision', 'PeerID', 'Phase', 'EstimatePeerID', 'CoordID'])

current_estimatePeerID_values = list(filter(lambda x: 'EstimatePeerID' in x, data_norm.columns.values))
current_coordID_values        = list(filter(lambda x: 'CoordID'        in x, data_norm.columns.values))
current_phase_value           = list(filter(lambda x: 'Phase'          in x, data_norm.columns.values))

coordID_values        = get_coordID_values(number_of_nodes)
EstimatePeerID_values = get_estimatePeerID_values(number_of_nodes)
phase_values          = ['Phase_1', 'Phase_2']

phase_colums          = data_norm.T.reindex(phase_values).T.fillna(0)
coordID_columns       = data_norm.T.reindex(coordID_values).T.fillna(0)
estimatePeerID_colums = data_norm.T.reindex(EstimatePeerID_values).T.fillna(0)

data_norm.drop(current_estimatePeerID_values +
               current_coordID_values        +
               current_phase_value,
               inplace=True,
               axis=1)

data_norm = pd.concat([data_norm, coordID_columns, estimatePeerID_colums, phase_colums], axis=1)
data_np   = data_norm.to_numpy()

delays        = []
mut_variables = []

for row in data_np:
    splited_data    = np.split(row, [2 + number_of_nodes, 2 + (number_of_nodes*2)])
    aggregated_data = np.concatenate((splited_data[0], splited_data[2]), axis=None)
    
    mut_variables.append([aggregated_data])
    delays.append([splited_data[1]])

delays = [[[ yNormalize(element, max_val_delay) for element in list] for list in matrix] for matrix in delays]

delays        = np.asarray(delays)
mut_variables = np.asarray(mut_variables)

mut_variables_train = mut_variables[:train_set_size]
mut_variables_test  = mut_variables[train_set_size:]

delays_train = delays[:train_set_size]
delays_test  = delays[train_set_size:]

print('Done')

In [0]:
number_of_nodes          = 40
algorithm_variables      = 5
mutation                 = 'merge'
matrix_rows, matrix_cols = 1, number_of_nodes*3 + 3

filepath = 'sample_data/snapshots_' + mutation + '_' + str(number_of_nodes)  + '_2.csv'
data     = load_data(filepath)

train_set_size = int(len(data) * 0.8)

#max_bandwidth  = data.Bandwidth.max()
#min_bandwidth  = data.Bandwidth.min()
#data.Bandwidth = data.Bandwidth.apply(xNormalize, args=(max_bandwidth, 0))

data_np = data.to_numpy()
delay   = data_np[:, (algorithm_variables + number_of_nodes):]

max_val_delay = np.amax(delay)
min_val_delay = np.amin(delay)

data_norm = pd.get_dummies(data, 
                           columns=['Decision', 'PeerID', 'CoordID'],
                           prefix=['Decision', 'PeerID', 'CoordID'])

current_coordID_values = list(filter(lambda x: 'CoordID' in x, data_norm.columns.values))
coordID_values         = get_coordID_values(number_of_nodes)
coordID_columns        = data_norm.T.reindex(coordID_values).T.fillna(0)

data_norm.drop(current_coordID_values, inplace=True, axis=1) 

data_norm = pd.concat([data_norm, coordID_columns], axis=1)
data_norm = desnormalize(data_norm)
data_np   = data_norm.to_numpy()

delays        = []
mut_variables = []

for row in data_np:
    splited_data    = np.split(row, [2 + number_of_nodes, 2 + (number_of_nodes*2)]) 
    aggregated_data = np.concatenate((splited_data[0], splited_data[2]), axis=None)
    
    mut_variables.append([aggregated_data])
    delays.append([splited_data[1]])

delays = [[[ yNormalize(element, max_val_delay) for element in list] for list in matrix] for matrix in delays]

delays        = np.asarray(delays)
mut_variables = np.asarray(mut_variables)

mut_variables_train = mut_variables[:train_set_size]
mut_variables_test  = mut_variables[train_set_size:]

delays_train = delays[:train_set_size]
delays_test  = delays[train_set_size:]


print('Done')

Done


In [0]:
C_mat = data_norm.corr()
fig   = plt.figure(figsize=(50,50))

sb.heatmap(C_mat, vmax = 1., square = True)
#plt.show()
plt.savefig('foo.png')

## Train, compile and fit model

In [0]:
!rm /content/Weights*.hdf5

keras.backend.clear_session()

batch_size = 16
sess       = tf.Session()  

K.set_session(sess)

# Activation function
activation = 'relu'

# Sequential model
model = Sequential()

# First layer
# The Input Layer :
input_nodes = len(mut_variables_train[0][0])
model.add(Dense(input_nodes, 
                kernel_initializer='normal', 
                input_shape=(1, matrix_cols), 
                activation=activation, 
                name="input_layer"))

# The Hidden Layers :
model.add(Dense(128, kernel_initializer='normal',activation=activation))
model.add(Dense(128, kernel_initializer='normal',activation=activation))
model.add(Dense(128, kernel_initializer='normal',activation=activation))

# The Output Layer :
model.add(Dense(number_of_nodes, 
                kernel_initializer='normal', 
                activation='linear', 
                name="output_layer"))

# Compile the network :

adam = Adam(lr=0.0001)
#adam = 'adam'
model.compile(loss='mean_absolute_error', 
              optimizer=adam, 
              metrics=['mean_absolute_error'])
model.summary()

checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint      = ModelCheckpoint(checkpoint_name, 
                                  monitor='val_loss', 
                                  verbose=1, 
                                  save_best_only = True, 
                                  mode ='auto')
callbacks_list = [checkpoint]

# Fit 
model.fit(mut_variables_train, 
          delays_train,
          batch_size=batch_size,
          epochs=150,
          validation_split=0.2,
          callbacks=callbacks_list)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (Dense)          (None, 1, 123)            15252     
_________________________________________________________________
dense_1 (Dense)              (None, 1, 128)            15872     
_________________________________________________________________
dense_2 (Dense)              (None, 1, 128)            16512     
_________________________________________________________________
dense_3 (Dense)              (None, 1, 128)            16512     
_________________________________________________________________
output_layer (Dense)         (None, 1, 40)             5160      
Total params: 69,308
Trainable params: 69,308
Non-trainable params: 0
_________________________________________________________________
Train on 44363 samples, validate on 11091 samples
Epoch 1/150
44363/44363 [==============================] - 9s 209us/step - loss: 0.0596 

In [0]:
sess.run(tf.global_variables_initializer())

from os      import listdir
from os.path import isfile, join

path  = '/content/'
files = [f for f in listdir(path) if isfile(join(path, f))]

weights_file = 'Weights-999--9.99999.hdf5' 
min_value    = 1000.0

for f in files:
    m = re.search('Weights-\d\d\d--(\d+\.\d+).hdf5', f)
    if m:
        value = float(m.group(1))
        if value < min_value:
            min_value = value
            weights_file = f
        
print(weights_file)
    
model.load_weights(weights_file)  
model.compile(loss='mean_absolute_error', 
              optimizer=Adam(lr=0.0001), 
              metrics=['mean_absolute_error'])

print('Done.')

Weights-150--0.01213.hdf5
Done.


In [0]:
# Get input and output layer name
for n in sess.graph.as_graph_def().node:
    if 'input_layer' in n.name:
          print(n.name)
    if 'output_layer' in n.name:
          print(n.name)

### **Evaluate predicitons**

In [0]:
predictions = model.predict(mut_variables_test)

for i in range(60,90):
    print(mut_variables_test[i][0])
    print(delays_test[i][0])
    print(predictions[i][0])
    print("----------" + str(i))


### **Test Centralized**

In [0]:
bandwidth = 500

test_data = [
    [
        [0,bandwidth,0,vote,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,peerID,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,coordID,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    ],
    [
        [0,bandwidth,vote,vote,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,peerID,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,coordID,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    ],
    [
        [0,bandwidth,vote,vote,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,peerID,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,coordID,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    ],
    [
        [0,bandwidth,vote,vote,0,0,0,0,0,0,0,0,0,0,0,0,0,vote,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,peerID,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,coordID,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    ],
    [
        [0,bandwidth,0,vote,0,0,0,0,0,0,0,0,0,0,0,0,vote,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,vote,0,0,peerID,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,coordID,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    ],
    [
        [maj,bandwidth,vote,vote,vote,vote,vote,vote,vote,vote,vote,0,0,vote,0,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,vote,0,0,peerID,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,coordID,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    ]
]

predictions = model.predict(np.asarray(test_data))

l = [2, 1, 2, 2, 2, 2]
i = 0

for li in l:
    print(predictions[i][0])
        
    print("- PEER " + str(li))
    i += 1
    

### **Test Ring**

In [0]:
bandwidth = 100

test_data = [
    [
        [0,bandwidth,0,vote,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,peerID,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,coordID,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    ],
    [
        [0,bandwidth,0,vote,vote,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,peerID,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,coordID,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    ],
    [
        [0,bandwidth,vote,vote,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,peerID,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,coordID,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    ],
    [
        [0,bandwidth,0,vote,vote,vote,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,peerID,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,coordID,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    ],
    [
        [0,bandwidth,vote,vote,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,vote,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,peerID,0,coordID,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    ],
    [
        [0,bandwidth,0,vote,vote,vote,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,peerID,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,coordID,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    ],
    [
        [0,bandwidth,vote,vote,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,vote,0,peerID,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,coordID,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    ],
    [
        [0,bandwidth,vote,vote,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,vote,vote,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,peerID,0,0,coordID,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    ],
    [
        [0,bandwidth,0,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,peerID,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,coordID,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    ],
    [
        [0,bandwidth,0,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,peerID,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,coordID,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    ],
    [ 
        [0,bandwidth,0,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,peerID,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,coordID,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    ],
    [ 
        [0,bandwidth,0,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,peerID,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,coordID,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    ],
    [ 
        [0,bandwidth,0,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,peerID,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,coordID,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    ],
    [ 
        [0,bandwidth,0,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,peerID,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,coordID,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    ],
    [ 
        [0,bandwidth,0,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,peerID,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,coordID,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    ],
    [ 
        [maj,bandwidth,0,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,peerID,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,coordID,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    ],
    [
        [maj,bandwidth,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,0,0,0,0,0,0,0,0,0,0,0,0,vote,vote,0,0,0,0,0,0,0,vote,vote,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,peerID,0,0,0,0,0,0,0,0,0,0,coordID,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    ],
    [
        [maj,bandwidth,0,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,vote,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,peerID,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,coordID,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    ]
]

predictions = model.predict(np.asarray(test_data))

l = [2, 3, 1, 4, 40, 3, 1, 39, 15, 16, 17, 18, 19, 20, 21, 22, 31, 5]
i = 0
for li in l:
    if li < 35 and li > 5:
        print(predictions[i][0][(li - 2):(li + 2)])
    elif li <= 5:
        print(predictions[i][0][38:], end='')
        print(predictions[i][0][:(li + 2)])
    else:
        print(predictions[i][0][(li - 2):], end='')
        print(predictions[i][0][:2])
        
    print("- PEER " + str(li))
    i += 1


In [0]:
# Use TF to save the graph model instead of Keras save model to load it in Golang
builder = tf.saved_model.builder.SavedModelBuilder("/content/drive/My Drive/datasets/mut_model_merge_40_5")  
# Tag the model, required for Go
builder.add_meta_graph_and_variables(sess, ["mut_tag"], clear_devices=True)  
builder.save()
# sess.close()